In [1]:
from replay_buffer_knapsack import *
from knapsack_env import *
import numpy as np
import tensorflow as tf
import time

In [2]:
ks = Knapsack(10, 1)

In [14]:
sess=tf.InteractiveSession() 
saver = tf.train.import_meta_graph('DQN_knapsack_model-learning_freq=4-450000.meta')
saver.restore(sess, tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./DQN_knapsack_model-learning_freq=4-460000


In [15]:
graph = tf.get_default_graph()
obs_t_ph = graph.get_tensor_by_name('obs_t_ph:0')
act_t_ph = graph.get_tensor_by_name('act_t_ph:0')
obs_tp1_ph = graph.get_tensor_by_name('obs_tp1_ph:0')

In [16]:
Q = graph.get_tensor_by_name('q_func/Q_output/BiasAdd:0')
target_Q = graph.get_tensor_by_name('target_q_func/Q_output/BiasAdd:0')

In [17]:
s = ks.reset()
s

array([ 0.37240609,  0.09792384,  0.03733614,  0.65127895,  0.77088659,
        0.24006911,  0.37159709,  0.46917842,  0.31136889,  0.9469951 ,
        0.47977943,  0.2053691 ,  0.68140345,  0.57474918,  0.06622397,
        0.88863718,  0.44074006,  0.70013001,  0.96598427,  0.62721508,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [18]:
ks.optimal_solution()[1]

array([ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.])

In [19]:
done = False
for i in range(5):
    print('i = ', i, ' - state:')
    print(np.reshape(s, [3, 10]))
    Q_val = sess.run(Q, {obs_t_ph: s[None]})
    print('Q_values:')
    print(Q_val[0])
    print('Playing ', np.argmax(Q_val[0]))
    s, rew, done, _ = ks.step(np.argmax(Q_val[0]))
    if done: break

print('Final state:')
print(np.reshape(s, [3, 10]))

print('\nAccuracy', 
      np.mean(1 - np.abs(np.reshape(s, [3, 10])[2] - ks.optimal_solution()[1])))

i =  0  - state:
[[ 0.37240609  0.09792384  0.03733614  0.65127895  0.77088659  0.24006911
   0.37159709  0.46917842  0.31136889  0.9469951 ]
 [ 0.47977943  0.2053691   0.68140345  0.57474918  0.06622397  0.88863718
   0.44074006  0.70013001  0.96598427  0.62721508]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]]
Q_values:
[ 0.53757703  0.26960829  0.22171406  0.83514005  0.90501523  0.41198078
  0.5652591   0.62704682  0.47168934  1.10922921]
Playing  9
i =  1  - state:
[[ 0.37240609  0.09792384  0.03733614  0.65127895  0.77088659  0.24006911
   0.37159709  0.46917842  0.31136889  0.9469951 ]
 [ 0.47977943  0.2053691   0.68140345  0.57474918  0.06622397  0.88863718
   0.44074006  0.70013001  0.96598427  0.62721508]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          1.        ]]
Q_values:
[ 0.54293728  0.2832469   0.217482    0.83753842  0.91502827  0.42984366
  0

In [20]:
accuracies = []
for i in range(10000):
    s = ks.reset()
    done = False
    for i in range(10):
        Q_val = sess.run(Q, {obs_t_ph: s[None]})
        s, rew, done, _ = ks.step(np.argmax(Q_val[0]))
        if done: break
    accuracies.append(np.mean(1 - \
                              np.abs(np.reshape(s, [3, 10])[2] - \
                                     ks.optimal_solution()[1])))

In [21]:
np.mean(accuracies)

0.7198